<a href="https://colab.research.google.com/github/tchotaneu/Computer_vision/blob/main/mainAPP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

charger le dossier depuis le drive google 

## initialisation 

In [ ]:
########################## Amine ######################### @author: Claude PAQUIE
import statistics
import time
import networkx as nx 
from random import Random
import numpy as np
from amine.amine import models
from amine.amine.datasets import Datasets
from amine.amine.module_detection import ModuleDetection
from amine.amine.scores import Scores

######################### Mane ########################  @author: sezin
import torch
import torch.nn as nn
from torch.autograd import Variable
import os
import MANE.without_attention.Node_Classification.generate_pairs as  generate_pairs
import random
import gc
import time
from sklearn import preprocessing
from collections import OrderedDict

######################### deplacement des fichiers et dossiers ###############
import shutil

######################### similarité cosinus  ########################
from scipy.spatial import distance

## definition des paramatres  de d' AMINE et Mane 

In [ ]:
#################### definition des parametres pour amine #################
class argumentAmine:    
     number_of_runs=3  #help="specifies the number of runs (default=1000)               type = int 
     nb_modules=1 #specifies the number of modules to generate (default=1)",    type = int 
     module_size=10 #specifies the size of each modules (default=10)                  type = int 
     target_module_size=None#help="specifies the expected size of each modules (default=None, i.e. no expected size)" type = int 
     network_size=1000 #  help="specifies the number of vertices in graph (default=1000)", type=int,
     removed_edges=0.0 # "proportion des arretes a supprime sur le graphe de donnees la valeur par defaut est  0.0
     verbose=False #  help="displays results on screen",
     outfile=None #help="name of the output file (default=no output)",
     graph_generation=["guyondata", "guyon", "batra", "gencat"]
     param_k=1#  ""K parameter used by Batra method to specify the average shortest path length
     param_mean_bg= 0 , #help="mean of the background used by Batra method (default=0)",
     param_mean_fg = 1   # help="mean of the foreground used by Batra method (default=1)", 
     param_std_bg=0 #="mean of the background used by Batra method (default=0)",
     param_std_fg=1 #mean of the foreground used by Batra method (default=1)"

#################### definition des parametres Mane #################

class  get_parser():
    read_pair=False
    input_graphs="MANE/data/networks/"
    input_pairs="MANE/data/pairs/"
    output="MANE/output/emb/"
    output_pairs="MANE/output/pairs/"
    dataset="Artificiel_Data/"
    nviews=2
    dimensions=64
    alpha=1.0
    beta= 1.0
    walk_length=10
    num_walks=5
    window_size=3
    p= 1.0
    q=1.0
    weighted=False
    unweighted=True
    directed=False
    undirected=True
    learning_rate=0.001
    negative_sampling=10.0
    batch_size=256
    epochs=1000# default 10
    cuda=True


## definition des fonction pour construire la deuxieme vue du dataset 

In [ ]:
# definition de la fonction pour construire un dataset artificielle a partir d'un autre 
def graph_artificiel1(G: nx.Graph, p_value:float =0.05) : 
    list_nodes=[] #liste des noeuds du graphe donnée en entree 
    nodes_pvalue=[] # liste des noeuds ayant une p_value inferieur à 0.05 en considerant leur poids 
    nodes_ww=[] # liste des noeuds sans leurs poids de p_valeur 
    edges_Gprime=[]  # liste des arretes du graphe G_prime 
    G_prime=nx.Graph() # initialisation du graphe G_prime 
    #print(list(G.nodes.data()))
    #construction de la liste des arretes du graphe G_prime 
    list_nodes=list(G.nodes.data()) # recuperation des l'ensemble des noeuds du graphes donnees en entrees 
        # selection des noeuds dont le poids est inferieur a la p_value
    for i in range (len(list_nodes)):
          if (list_nodes[i][1]['weight'])<p_value:
            nodes_pvalue.append(list_nodes[i])
            nodes_ww.append(list_nodes[i][0])
    G_prime.add_nodes_from(nodes_pvalue) #affectation des noeuds au graph G_prime 
        #construction de la liste des arretes du graphe G_prime 
    a=0 
    #print(nodes_ww)
    for i in range(len(nodes_ww)-1):
        a=i+1
        while a<len(nodes_ww):
            edges_Gprime.append((nodes_ww[i],nodes_ww[a],{'weight': None}))
            a=a+1
    #print(edges_Gprime)
    G_prime.add_edges_from(edges_Gprime)
    return  Datasets.init_graph(G_prime, edge_weight=None, default_groups="truehit")




# definition de la fonction pour construire un dataset artificielle a partir d'un autre 
def graph_artificiel2(G: nx.Graph, p_value:float =0.05) : 
    list_nodes=[] #liste des noeuds du graphe donnée en entree 
    nodes_pvalue=[] # liste des noeuds ayant une p_value inferieur à 0.05 en considerant leur poids 
    nodes_pvalueGrand=[] # liste des noeuds ayant une p_value inferieur à 0.05 en considerant leur poids 
    nodes_ww=[] # liste des noeuds sans leurs poids de p_valeur 
    edges_Gprime=[]  # liste des arretes du graphe G_prime 
    G_prime=nx.Graph() # initialisation du graphe G_prime 
    #print(list(G.nodes.data()))
    #construction de la liste des arretes du graphe G_prime 
    list_nodes=list(G.nodes.data()) # recuperation des l'ensemble des noeuds du graphes donnees en entrees 
        # selection des noeuds dont le poids est inferieur a la p_value
    for i in range (len(list_nodes)):
          if (list_nodes[i][1]['weight'])<p_value:
            nodes_pvalue.append(list_nodes[i])
            nodes_ww.append(list_nodes[i][0])
          else:
            nodes_pvalueGrand.append(list_nodes[i])
    G_prime.add_nodes_from(nodes_pvalue) #affectation des noeuds au graph G_prime 
    G_prime.add_nodes_from(nodes_pvalueGrand)
        #construction de la liste des arretes du graphe G_prime 
    a=0 
    #print(nodes_ww)
    for i in range(len(nodes_ww)-1):
        a=i+1
        while a<len(nodes_ww):
            edges_Gprime.append((nodes_ww[i],nodes_ww[a],{'weight': None}))
            a=a+1
    #print(edges_Gprime)
    G_prime.add_edges_from(edges_Gprime)
    return  Datasets.init_graph(G_prime, edge_weight=None, default_groups="truehit")

##  fonction pour deplacer les fichiers entre les Amine et Mane

In [ ]:

def deplacer_fichier(fichier, dossier_destination):
    try:
        # Vérifier si le fichier existe
        if os.path.isfile(fichier):
            # Vérifier si le dossier de destination existe
            if os.path.isdir(dossier_destination):
                # Obtenir le nom du fichier
                nom_fichier = os.path.basename(fichier)
                
                # Déplacer le fichier vers le dossier de destination en écrasant si nécessaire
                shutil.move(fichier, os.path.join(dossier_destination, nom_fichier))
                print("Le fichier a été déplacé avec succès.")
            else:
                print("Le dossier de destination n'existe pas.")
        else:
            print("Le fichier spécifié n'existe pas.")
    except Exception as e:
        print("Une erreur s'est produite lors du déplacement du fichier :", str(e))



def create_folder(nom_dossier,num):
    new_name=nom_dossier+str(num)
    if not os.path.exists(new_name):
    # Créer le dossier
      os.makedirs(new_name)
      print("Le dossier a été créé avec succès.")
    else:
      print("Le dossier existe déjà.")
    return new_name

def move_file(fichier, dossier):
    # Vérifier si le fichier existe
    if os.path.exists(fichier):
        # Vérifier si le dossier de destination existe
        if os.path.exists(dossier):
            # Déplacer le fichier dans le dossier de destination
            shutil.move(fichier, dossier)
            print("Le fichier a été déplacé avec succès dans le dossier.")
        else:
            print("attention le dosssier n'exite pas .")
    else:
        print(" attention le fichier n'existe pas.")


def deplacer_fichiers(dossier_source, dossier_destination):
    # Parcours de tous les fichiers du dossier source
    for fichier in os.listdir(dossier_source):
        chemin_fichier_source = os.path.join(dossier_source, fichier)
        if os.path.isfile(chemin_fichier_source):
            chemin_fichier_destination = os.path.join(dossier_destination, fichier)
            shutil.move(chemin_fichier_source, chemin_fichier_destination)



def transfer_file(dossier_source,dossier_destination):

    # Parcours de tous les fichiers du dossier source
    for fichier in os.listdir(dossier_source):
        chemin_fichier_source = os.path.join(dossier_source, fichier)
        if os.path.isfile(chemin_fichier_source):
            chemin_fichier_destination = os.path.join(dossier_destination, fichier)
            shutil.move(chemin_fichier_source, chemin_fichier_destination)


def delete_folder(nom_dossier): 
    # Vérifier si le dossier existe
    if os.path.exists(nom_dossier):
        # Supprimer le dossier et son contenu
        shutil.rmtree(nom_dossier)
        print("Le dossier a été supprimé avec succès.")
    else:
        print("Le dossier n'existe pas.")

## definition de la classe Mane

In [ ]:

class Mane(nn.Module):  # ready for cluster, no cache cleaning or loop shortening
    def __init__(self, params, len_common_nodes, embed_freq, batch_size, negative_sampling_size=10):
        super(Mane, self).__init__()
        self.n_embedding = len_common_nodes
        self.embed_freq = embed_freq
        self.num_net = params.nviews
        self.negative_sampling_size = negative_sampling_size
        self.node_embeddings = nn.ModuleList()
        self.neigh_embeddings = nn.ModuleList()
        self.embedding_dim = params.dimensions
        self.device = params.device
        for n_net in range(self.num_net):  # len(G)
            self.node_embeddings.append(nn.Embedding(len_common_nodes, self.embedding_dim))
            self.neigh_embeddings.append(nn.Embedding(len_common_nodes, self.embedding_dim))

        self.batch_size = batch_size

    def forward(self, count, shuffle_indices_nets, nodes_idx_nets, neigh_idx_nets, hyp1, hyp2):
        cost1 = [nn.functional.logsigmoid(torch.bmm(self.neigh_embeddings[i](Variable(torch.LongTensor(
            neigh_idx_nets[i][shuffle_indices_nets[i][count:count + self.batch_size]]).to(self.device))).unsqueeze(
            2).view(
            len(shuffle_indices_nets[i][count:count + self.batch_size]), -1,
            self.embedding_dim), self.node_embeddings[i](Variable(
            torch.LongTensor(nodes_idx_nets[i][shuffle_indices_nets[i][count:count + self.batch_size]]).to(
                self.device))).view(
            len(shuffle_indices_nets[i][count:count + self.batch_size]), -1).unsqueeze(
            2))).squeeze().mean() + nn.functional.logsigmoid(torch.bmm(self.neigh_embeddings[i](
            self.embed_freq.multinomial(
                len(shuffle_indices_nets[i][count:count + self.batch_size]) * self.neigh_embeddings[i](Variable(
                    torch.LongTensor(
                        neigh_idx_nets[i][shuffle_indices_nets[i][count:count + self.batch_size]]).to(
                        self.device))).unsqueeze(
                    2).view(len(shuffle_indices_nets[i][count:count + self.batch_size]), -1,
                            self.embedding_dim).size(1) * self.negative_sampling_size, replacement=True).to(
                self.device)).view(
            len(shuffle_indices_nets[i][count:count + self.batch_size]), -1,
            self.embedding_dim).neg(), self.node_embeddings[i](Variable(
            torch.LongTensor(nodes_idx_nets[i][shuffle_indices_nets[i][count:count + self.batch_size]]).to(
                self.device))).view(
            len(shuffle_indices_nets[i][count:count + self.batch_size]), -1).unsqueeze(2))).squeeze().sum(1).mean(0) for
                 i in range(self.num_net)]

        # First order collaboration
        cost2 = [[hyp1 * (nn.functional.logsigmoid(torch.bmm(self.node_embeddings[j](Variable(torch.LongTensor(
            nodes_idx_nets[i][shuffle_indices_nets[i][count:count + self.batch_size]]).to(self.device))).unsqueeze(
            2).view(
            len(shuffle_indices_nets[i][count:count + self.batch_size]), -1, self.embedding_dim),
            self.node_embeddings[i](Variable(torch.LongTensor(
                nodes_idx_nets[i][shuffle_indices_nets[i][
                                  count:count + self.batch_size]]).to(self.device))).view(
                len(shuffle_indices_nets[i][
                    count:count + self.batch_size]), -1).unsqueeze(
                2))).squeeze().mean() + nn.functional.logsigmoid(
            torch.bmm(self.node_embeddings[j](self.embed_freq.multinomial(
                len(shuffle_indices_nets[i][count:count + self.batch_size]) * self.node_embeddings[j](Variable(
                    torch.LongTensor(
                        nodes_idx_nets[i][shuffle_indices_nets[i][count:count + self.batch_size]]).to(
                        self.device))).unsqueeze(
                    2).view(len(shuffle_indices_nets[i][count:count + self.batch_size]), -1, self.embedding_dim).size(
                    1) * self.negative_sampling_size,
                replacement=True).to(self.device)).view(len(shuffle_indices_nets[i][count:count + self.batch_size]), -1,
                                                        self.embedding_dim).neg(), self.node_embeddings[i](Variable(
                torch.LongTensor(
                    nodes_idx_nets[i][shuffle_indices_nets[i][count:count + self.batch_size]]).to(self.device))).view(
                len(shuffle_indices_nets[i][count:count + self.batch_size]), -1).unsqueeze(2))).squeeze().sum(1).mean(
            0))
                  for i in range(self.num_net) if i != j] for j in range(self.num_net)]

        # Second order collaboration

        cost3 = [[hyp2 * (nn.functional.logsigmoid(torch.bmm(self.neigh_embeddings[j](Variable(torch.LongTensor(
            neigh_idx_nets[i][shuffle_indices_nets[i][count:count + self.batch_size]]).to(self.device))).unsqueeze(
            2).view(
            len(shuffle_indices_nets[i][count:count + self.batch_size]), -1, self.embedding_dim),
            self.node_embeddings[i](Variable(torch.LongTensor(
                nodes_idx_nets[i][shuffle_indices_nets[i][
                                  count:count + self.batch_size]]).to(self.device))).view(
                len(shuffle_indices_nets[i][
                    count:count + self.batch_size]), -1).unsqueeze(
                2))).squeeze().mean() + nn.functional.logsigmoid(
            torch.bmm(self.neigh_embeddings[j](self.embed_freq.multinomial(
                len(shuffle_indices_nets[i][count:count + self.batch_size]) * self.neigh_embeddings[j](Variable(
                    torch.LongTensor(
                        neigh_idx_nets[i][shuffle_indices_nets[i][count:count + self.batch_size]]).to(
                        self.device))).unsqueeze(
                    2).view(len(shuffle_indices_nets[i][count:count + self.batch_size]), -1, self.embedding_dim).size(
                    1) * self.negative_sampling_size,
                replacement=True).to(self.device)).view(len(shuffle_indices_nets[i][count:count + self.batch_size]), -1,
                                                        self.embedding_dim).neg(), self.node_embeddings[i](Variable(
                torch.LongTensor(
                    nodes_idx_nets[i][shuffle_indices_nets[i][count:count + self.batch_size]]).to(self.device))).view(
                len(shuffle_indices_nets[i][count:count + self.batch_size]), -1).unsqueeze(2))).squeeze().sum(1).mean(
            0))
                  for i in range(self.num_net) if i != j] for j in range(self.num_net)]

        sum_cost2 = []
        [[sum_cost2.append(j) for j in i] for i in cost2]

        sum_cost3 = []
        [[sum_cost3.append(j) for j in i] for i in cost3]

        return -(torch.mean(torch.stack(cost1)) + sum(sum_cost2) / len(sum_cost2) + sum(sum_cost3) / len(sum_cost3)) / 3



def read_graphs(current_path, n_views):
    """
        Read graph/network data for each view from an adjlist (from networkx package)

    :param current_path: path for graph data
    :param n_views: number of views
    :return: A list of graphs
    """
    entries = os.listdir(current_path)
    G = []
    if len(entries) != n_views:
        print("WARNING: Number of networks in the folder is not equal to number of views setting.")
    for n_net in range(n_views):
        G.append(nx.read_adjlist(current_path + entries[n_net]))
        print("Network ", (n_net + 1), ": ", entries[n_net])
    return G


def read_word2vec_pairs(current_path, nviews):
    """

    :param current_path: path for two files, one keeps only the node indices, the other keeps only the neighbor node
    indices of already generated pairs (node,neighbor), i.e, node indices and neighbor indices are kept separately.
    method "construct_word2vec_pairs" can be used to obtain these files.
    :E.g.:

      for pairs (9,2) (4,5) (8,6) one file keeps 9 4 8 the other file keeps 2 5 6.

    :param nviews: number of views
    :return: Two lists for all views, each list keeps the node indices of node pairs (node, neigh).
    nodes_idx_nets for node, neigh_idx_nets for neighbor
    """

    nodes_idx_nets = []
    neigh_idx_nets = []

    for n_net in range(nviews):
        neigh_idx_nets.append(np.loadtxt(current_path + "/neighidxPairs_" + str(n_net + 1) + ".txt"))
        nodes_idx_nets.append(np.loadtxt(current_path + "/nodesidxPairs_" + str(n_net + 1) + ".txt"))
    return nodes_idx_nets, neigh_idx_nets


def degree_nodes_common_nodes(G, common_nodes, node2idx):
    """
    Assigns scores for negative sampling distribution
    """
    degrees_idx = dict((node2idx[v], 0) for v in common_nodes)
    multinomial_nodesidx = []
    for node in common_nodes:
        degrees_idx[node2idx[node]] = sum([G[n].degree(node) for n in range(len(G))])
    for node in common_nodes:
        multinomial_nodesidx.append(degrees_idx[node2idx[node]] ** (0.75))

    return multinomial_nodesidx


def mainMane():
    """
    Initialize parameters and train
    """

    params = get_parser()
    print(params)

    if torch.cuda.is_available() and not params.cuda:
        print("WARNING: You have a CUDA device, you may try cuda with --cuda")
    device = 'cuda:0' if torch.cuda.is_available() and params.cuda else 'cpu'
    params.device = device
    print("Running on device: ", device)
    G = read_graphs(params.input_graphs + params.dataset, params.nviews)
    common_nodes = sorted(set(G[0]).intersection(*G))
    print('Number of common/core nodes in all networks: ', len(common_nodes))
    node2idx = {n: idx for (idx, n) in enumerate(common_nodes)}
    print("node2idx",node2idx)
    idx2node = {idx: n for (idx, n) in enumerate(common_nodes)}

    if params.read_pair:

        nodes_idx_nets, neigh_idx_nets = read_word2vec_pairs("MANE/"+params.input_pairs + params.dataset, params.nviews)

    else:
        nodes_idx_nets = []
        neigh_idx_nets = []
        for n_net in range(params.nviews):
            view_id = n_net + 1
            print("View ", view_id)

            nodes_idx, neigh_idx = generate_pairs.construct_word2vec_pairs(G[n_net], view_id, common_nodes, params.p,
                                                                           params.q, params.window_size,
                                                                           params.num_walks,
                                                                           params.walk_length,
                                                                           params.output_pairs , #+ params.dataset
                                                                           node2idx)

            nodes_idx_nets.append(nodes_idx)
            neigh_idx_nets.append(neigh_idx)

    multinomial_nodes_idx = degree_nodes_common_nodes(G, common_nodes, node2idx)

    embed_freq = Variable(torch.Tensor(multinomial_nodes_idx))

    model = Mane(params, len(common_nodes), embed_freq, params.batch_size)
    model.to(device)

    epo = 0
    min_pair_length = nodes_idx_nets[0].size
    for n_net in range(params.nviews):
        if min_pair_length > nodes_idx_nets[n_net].size:
            min_pair_length = nodes_idx_nets[n_net].size
    print("Total number of pairs: ", min_pair_length)
    print("Training started! \n")

    while epo <= params.epochs - 1:
        start_init = time.time()

        epo += 1
        optimizer = torch.optim.Adam(model.parameters(), lr=params.learning_rate)
        running_loss = 0
        num_batches = 0
        shuffle_indices_nets = []
        fifty = False

        for n_net in range(params.nviews):
            shuffle_indices = [x for x in range(nodes_idx_nets[n_net].size)]
            random.shuffle(shuffle_indices)
            shuffle_indices_nets.append(shuffle_indices)
        for count in range(0, min_pair_length, params.batch_size):
            optimizer.zero_grad()
            loss = model(count, shuffle_indices_nets, nodes_idx_nets, neigh_idx_nets, params.alpha, params.beta)
            loss.backward()
            optimizer.step()
            running_loss += loss.detach().item()
            num_batches += 1
            if int(num_batches % 100) == 0:
                print(num_batches, " batches completed\n")
            elif not fifty and (count / min_pair_length) * 100 > 50:
                print("############# 50% epoch is completed #################\n")
                fifty = True
            torch.cuda.empty_cache()
            gc.collect()

        total_loss = running_loss / (num_batches)
        elapsed = time.time() - start_init
        print('epoch=', epo, '\t time=', elapsed, ' seconds\t total_loss=', total_loss)

    concat_tensors = model.node_embeddings[0].weight.detach().cpu()
    print('Embedding of view ', 1, ' ', concat_tensors)

    for i_tensor in range(1, model.num_net):
        print('Embedding of view ', (i_tensor + 1), ' ', model.node_embeddings[i_tensor].weight.detach().cpu())
        concat_tensors = torch.cat((concat_tensors, model.node_embeddings[i_tensor].weight.detach().cpu()), 1)

    emb_file = params.output + params.dataset + "Embedding_" + "concatenated_without_attention" + '_epoch_' + str(
        epo) + "_" + ".txt"
    embed_result = np.array(concat_tensors)
    fo = open(emb_file, 'a+')
    for idx in range(len(embed_result)):
        word = (idx2node[idx])
        fo.write(word + ' ' + ' '.join(
            map(str, embed_result[idx])) + '\n')
    fo.close()




### affichage des graphe des courbe d'apprentissage de former l'embbeding 

## definition d'Amine 

In [ ]:

arg=argumentAmine()
destinationPair ="MANE/output/pairs"
destinationGrap ="MANE/output/pairs"
dossierdetraitement="MANE/data/networks/Artificiel_Data"
sourcePair="output/pairs"
__name__= "main"
if __name__ == "main":
    # Entry point.
    #arg = parse_arguments()

    # Use Node2vec model
    model = models.Node2vec()

    # Use aggregation zscore as fitness function.
    fitness_fun = lambda the_graph, clus: Scores.aggregation_from_pvalue(
        the_graph, clus, "weight"
    )

    # set characteristic of dense network
    P_PROB = 0.09
    Q_PROB = 0.70
    NB_INITIAL_NODES = 3

    f1_scores = []
    if arg.outfile:
        outfile = open(arg.outfile, "w")
        outfile.write("#graph,time(s),nb found,real size,true hits,pvalue\n")
    for ctr in range(arg.number_of_runs):
        if arg.graph_generation == "guyondata":
            G = Datasets.get_guyon_graph(ctr + 1)
        elif arg.graph_generation == "gencat":
            G = Datasets.get_gencat_graph(
                arg.network_size,
                arg.network_size * 16,
                arg.nb_modules,
                arg.nb_modules * [arg.module_size],
                ctr,
            )
        else:
            G = Datasets.get_scale_free_graph(
                arg.network_size,
                NB_INITIAL_NODES,
                arg.nb_modules,
                arg.module_size,
                P_PROB,
                Q_PROB,
                ctr,
            )
            if arg.removed_edges > 0:
                nbtoremove = int(G.number_of_edges() * arg.removed_edges)
                rng = Random(ctr)
                edges_to_remove = rng.sample(G.edges, nbtoremove)
                G.remove_edges_from(edges_to_remove)
                G.graph["nb_edges"] = G.number_of_edges()

        
        # contrution du graphe seecondaire 
        G_p=graph_artificiel2(G)  # appel de la fonction pour contruire le graphe 
        # nommage de fichier pour stocke les graphes  ** numeroRun+sub1.txt
        graphe_vue1=str(ctr+1)+"_sub1.txt" 
        graphe_vue2=str(ctr+1)+"_sub2.txt" # nommage de fichier pour stocke les graphes  ** numeroRun+sub2.txt
        #ecriture du graphe dans un fichier 
        nx.write_adjlist( G, graphe_vue1) # sauvegarde du graphe dans un fichier 
        nx.write_adjlist( G_p, graphe_vue2)
        # deplacement dans le dossier data de Mane
        move_file(graphe_vue1,dossierdetraitement) 
        move_file(graphe_vue2,dossierdetraitement) 
        # appel de la fonction Mane 
        mainMane()
        # creer le dossier de stokage de des pairs creer 
        destPair=create_folder(destinationPair,str(ctr+1))
        destGrap=create_folder(destinationGrap,str(ctr+1))
        # deplacer les fichier du dossier output/pair dans un autre dossier de stockage pour un calcul ulterieur 
        deplacer_fichiers(sourcePair,destPair)
        #deplacer les fichiers des datatratemengt  traité vers un autre dossier 
        deplacer_fichiers(dossierdetraitement,destGrap)
        # recuperation de du fichier embedding 
        
        
        # initialize the model
        #model.init(G)

        # call module detection method
        start_time = time.perf_counter()
        module_detection = ModuleDetection(
            G, model, fitness_fun, background_correction=True
        )

        # do the prediction
        results = module_detection.predict_modules(
            max_nb_modules=arg.nb_modules, cutoff=0.05
        )
        end_time = time.perf_counter()
        pvalue = 0
        if results:
            pvalue = results[0][2]
        truehits = Datasets.get_groups(G)
        print("truehits", truehits)

        pred = set()
        nb_clust = 0
        for cl in results:
            if arg.target_module_size:
                if not pred:
                    pred.update(cl[0])
                    continue
                if len(pred) < arg.target_module_size:
                    if abs(arg.target_module_size - len(pred)) > abs(
                        arg.target_module_size - len(pred) - len(cl[0])
                    ):
                        pred.update(cl[0])
                        continue
                    else:
                        break
                else:
                    break
            else:
                nb_clust += 1
                if nb_clust > arg.nb_modules:
                    break
                pred.update(cl[0])

        # merge the content of the true hits in a set
        th = set()
        for key, value in truehits.items():
            th.update(value)

        nb_pred = len(pred)
        nb_th = len(th)
        nb_found = len(pred & th)

        if arg.outfile:
            perf = end_time - start_time
            outfile.write(
                f"{ctr + 1},{perf:.2f},{nb_pred},{nb_th},{nb_found},{pvalue}\n"
            )
            outfile.flush()
        if not arg.verbose:
            continue
        # calculate the F1 score
        res = []
        f1_score = Scores.measure_f1(G, th, pred)
        res.append((len(pred), len(pred & th), f1_score))

        f1_scores.append(f1_score)

        # compute the stats
        quartiles = np.percentile(f1_scores, [25, 50, 75])
        minim = min(f1_scores)
        maxim = max(f1_scores)
        iqr = quartiles[2] - quartiles[0]
        high = min(maxim, quartiles[2] + iqr * 1.5)
        low = max(minim, quartiles[0] - iqr * 1.5)
        out_high = [str(f"{x:.5f}") for x in f1_scores if x > high]
        out_low = [str(f"{x:.5f}") for x in f1_scores if x < low]
        print(
            f"{ctr + 1} len={nb_pred} true_hits={nb_th} found={nb_found} f1_score={f1_scores[-1]:.5f}, mean={statistics.mean(f1_scores):.5f}, variance={statistics.pstdev(f1_scores):.5f}, Q25={quartiles[0]:.5f}, Q50={quartiles[1]:.5f}, Q75={quartiles[2]:.5f}, low={low:.5f}, high={high:.5f}, out_low=[{','.join(out_low)}], out_high=[{','.join(out_high)}] pvalue={pvalue}"
        )
    if arg.outfile:
        outfile.close()


## rechercher de la similarité entre les noeuds 

In [ ]:

def ajouter_element(ensemble, element):
    ensemble.add(element)

fichier = "MANE/output/emb/Artificiel_Data/Embedding_concatenated_without_attention_epoch_1000_.txt"
vectors = np.loadtxt(fichier)
labels = vectors[:, 0]  # Extraction de la première colonne (labels)
vectors = vectors[:, 1:]  # Extraction des autres colonnes (composantes vectorielles)
for id_noeud in range(1000):
    reference_vector = vectors[id_noeud ]
    # Calcul de la similarité cosinus
    similarities = []
    for vector in vectors:
        similarity = 1 - distance.cosine(reference_vector, vector)
        similarities.append(similarity)

    # Sélection des indices des vecteurs avec une similarité supérieure à 0.95
    
    # Sélection des indices des vecteurs avec une similarité supérieure à 0.95
    selected_indices = [i for i, similarity in enumerate(similarities) if similarity > 0.65]
    selected_indices = selected_indices[:10]  # Sélection des 10 premiers indices

    # Sélection des labels et des vecteurs correspondants
    selected_labels = labels[selected_indices]
    selected_vectors = vectors[selected_indices]
    mon_ensemble = []
    # Affichage des vecteurs similaires avec leurs labels*
    
    for label, vector in zip(selected_labels, selected_vectors):
       # print("Label:", str(int(label)))
       mon_ensemble.append(int(label))
        #ajouter_element(mon_ensemble, label)
    if len(mon_ensemble)>2 :
        print("######################################")
        print(mon_ensemble)  # Affiche : {1, 2, 3, 4}

        #print("Vecteur:", vector)